<a href="https://colab.research.google.com/github/ayushbond007/BOOK-recommender-system/blob/main/Simple_Popularity_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import pandas as pd
import numpy as np

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# tools
import scipy
import math
import random
import sklearn
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split


finalbooks = pd.read_csv('../tmp/finalbooks.csv')
ratings = pd.read_csv('../tmp/finalratings.csv')
train = pd.read_csv('../tmp/train.csv')
test = pd.read_csv('../tmp/test.csv')

In [2]:
from google.colab import drive 
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
finalbooks=pd.read_csv("/content/drive/MyDrive/BOOK RECOMENDATION /finalfor modelling.csv")
ratings= pd.read_csv("/content/drive/MyDrive/BOOK RECOMENDATION /new_rating.csv")

In [ ]:
 train, test = train_test_split(ratings,
                               stratify=ratings['newuser_id'], 
                               test_size=0.20,
                               random_state=42)

In [ ]:
test['newbookid'].value_counts()

In [ ]:

def dcg_k(r, k):  
    r = np.asfarray(r)[:k]
    return np.sum(2**r / np.log2(np.arange(2, r.size + 2)))      

    
def ndcg_k(r, k):
    dcg_max = dcg_k(sorted(r, reverse=True), k)
    if not dcg_max:
        return 0.
    return dcg_k(r, k) / dcg_max

def mean_ndcg(rs):
    return np.mean([ndcg_k(r, len(r)) for r in rs])

In [ ]:
def precision_k(r, k):
    assert k >= 1
    r = np.asarray(r)[:k] != 0
    if r.size != k:
        raise ValueError('Relevance score length < k')
    return np.mean(r)

def average_precision(r):

    r = np.asarray(r) != 0
    out = [precision_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)


def mean_average_precision(rs):
    return np.mean([average_precision(r) for r in rs]) 


In [ ]:
def rmse(y,h):
    a = y-h
    return np.sqrt(sum(a**2)/len(a))

In [ ]:
# Distribution of the # of Ratings per Book
facet, axes = plt.subplots(1, 1, figsize=(10, 3))
n, bins, patches = plt.hist(ratings['newbookid'].value_counts(), 200, facecolor='blue', alpha=0.5) #, log = True)   
plt.title('Distribution of # of reviews per Book in Dataset (mean = 597 reviews / max = 22806 / min = 8 )')
plt.show()

In [ ]:
## DEFINING THE TAIL
tailcomp = ratings.groupby(by= 'newbookid', as_index=False).agg({'rating':pd.Series.count}).sort_values(by = 'rating', ascending = False)
tot = sum(tailcomp['rating'])
tailcomp['popshare']= [x/tot for x in tailcomp['rating']]
tailcomp['popshare']= tailcomp['popshare'].cumsum()
tailcomp['category']= ['Head' if x<0.95 else "Tail" for x in tailcomp['popshare']]

#finalbooks.loc[finalbooks.popshare >= 0.8].describe()
tail = tailcomp.loc[tailcomp.popshare >= 0.95]
tail

In [ ]:
# Distribution of the # of Ratings per Book
facet, axes = plt.subplots(1, 1, figsize=(10, 3))
n, bins, patches = plt.hist(ratings['newbookid'].value_counts(), 200, facecolor='blue', alpha=0.5) #, log = True)   
plt.title('Distribution of # of reviews per Book in Dataset (mean = 597 reviews / max = 22806 / min = 8 )')
plt.show()

In [ ]:
## BASELINE APPROACH: POPULARITY MODEL

popular = finalbooks.groupby('newbookid')['ratings_count','average_rating'].sum().sort_values(by = 'ratings_count',ascending=False).reset_index()
popular.head(5)

In [ ]:
poprank = test.merge(popular,on = 'newbookid')
poprank['pred']= np.round(poprank['average_rating'])
poprank['ratingYN']= [1 if x>= 4 else 0 for x in poprank['rating']]


poprank = poprank.sort_values(by=['newuser_id', 'ratings_count'], ascending=False)
poprank.head(5)

In [ ]:

poplista = []

for i in range(15000):
  a = poprank.loc[poprank.newuser_id == i+1]['rating'].tolist()
  poplista.append(a)
  if (i+1)%1000 == 0: print("done: ", i+1)
   



In [ ]:
top = poprank.sort_values('ratings_count',ascending = False).groupby('newuser_id').head(10)
top

In [ ]:
b = np.array([ndcg_k(r, len(r)) for r in poplista])

facet, axes = plt.subplots(1, 1, figsize=(10, 3))
n, bins, patches = plt.hist(b, 200, facecolor='blue', alpha=0.5) #, log = True)   
plt.title('Distribution of NDGC among Users for the Popularity model')
plt.show()

# [ndcg_k(r, len(r)) for r in poplista]
d = b[b == 1]
sum(d)/15000

In [ ]:

# print('Popularity Model MAP: ', mean_average_precision(poplistb))
print('(1) Popularity Model RMSE: ', np.round(rmse(poprank['rating'],poprank['average_rating']), decimals=3))
print('(2) Popularity Model NDCG: ', np.round(mean_ndcg(poplista), decimals=3))
print("(2) Median NDCG: ", np.round(np.median(b), decimals=3))
print("(2) Share of NDCG =1 among Users: ", np.round(sum(d)/15000, decimals=3))
#print('(3) Popularity Model Div10 Score: ',np.round(sum(np.in1d(top.newbookid, tail.newbookid))/len(top), decimals=2))
#print('(3) Popularity Model Div50 Score: ',np.round(sum(np.in1d(top.newbookid, tail.newbookid))/len(top), decimals=2))

In [ ]:
popranktrain = test.merge(popular,on = 'newbookid')
popranktrain['pred']= np.round(poprank['average_rating'])

In [ ]:
poplisttrain = []
for i in range(15000):
    a = popranktrain.loc[popranktrain.newuser_id == i+1]['rating'].tolist()
    poplisttrain.append(a)
    if (i+1)%1000 == 0: print("done: ", i+1)

In [ ]:
print('(1) Pop Train Model RMSE: ', np.round(rmse(popranktrain['pred'],popranktrain['rating']), decimals=3))
print('(2) Pop Train Model NDCG: ', np.round(mean_ndcg(poplisttrain), decimals=3))